# Databricks EDA — Squeeze Analytics (Sydney time)

Assumes your SQLite tables have been loaded into Delta tables (Unity Catalog) as in `copy_into_manual_upload_volume.ipynb`.

This notebook focuses on:
- quick table sanity checks
- timestamp normalization (epoch ms → timestamp)
- converting to **Australia/Sydney** (AEST/AEDT DST-aware)
- basic distributions across alerts
- **OHLC data quality** (gaps/duplicates/invalid bars)
- generating a reusable **clean universe** table for backtests


In [ ]:
%sql
-- ---- CONFIG ----
USE CATALOG `workspace`;
USE SCHEMA `squeeze`;
SHOW TABLES;


## Row counts

In [ ]:
%sql
SELECT 'ohlc' AS table, COUNT(*) AS n FROM ohlc
UNION ALL SELECT 'alerts', COUNT(*) FROM alerts
UNION ALL SELECT 'trade_plans', COUNT(*) FROM trade_plans
UNION ALL SELECT 'backtest_trades', COUNT(*) FROM backtest_trades
UNION ALL SELECT 'backtest_results', COUNT(*) FROM backtest_results
UNION ALL SELECT 'snapshot_cache', COUNT(*) FROM snapshot_cache
UNION ALL SELECT 'market_cap_cache', COUNT(*) FROM market_cap_cache
ORDER BY n DESC;


## Timestamp normalization patterns (epoch ms → UTC timestamp → Sydney time)

In Spark/Databricks:
- `to_timestamp(ts/1000)` converts epoch seconds to a timestamp
- `from_utc_timestamp(..., 'Australia/Sydney')` converts a UTC timestamp to Sydney local time

These functions are DST-aware.

In [ ]:
%sql
-- Alerts: ts and created_ts in Sydney time
SELECT
  exchange, symbol, signal, source_tf,
  ts,
  to_timestamp(ts/1000) AS ts_utc,
  from_utc_timestamp(to_timestamp(ts/1000), 'Australia/Sydney') AS ts_sydney,
  created_ts,
  from_utc_timestamp(to_timestamp(created_ts/1000), 'Australia/Sydney') AS created_ts_sydney
FROM alerts
ORDER BY ts DESC
LIMIT 50;


## Alert distributions

In [ ]:
%sql
SELECT signal, COUNT(*) AS n
FROM alerts
GROUP BY signal
ORDER BY n DESC
LIMIT 50;


In [ ]:
%sql
SELECT source_tf, COUNT(*) AS n
FROM alerts
GROUP BY source_tf
ORDER BY n DESC
LIMIT 50;


## Day-of-week and hour-of-day in Sydney time

In [ ]:
%sql
WITH x AS (
  SELECT
    from_utc_timestamp(to_timestamp(ts/1000), 'Australia/Sydney') AS ts_syd
  FROM alerts
)
SELECT
  date_format(ts_syd, 'E') AS dow_syd,
  hour(ts_syd) AS hour_syd,
  COUNT(*) AS n
FROM x
GROUP BY 1,2
ORDER BY 1,2;


# OHLC Data Quality → Clean Universe
We’ll compute integrity metrics per `(exchange, symbol, interval)` and write a `clean_universe` table.

Checks include:
- duplicate bars (same open_time)
- time gaps (open_time not equal to previous + expected interval ms)
- invalid OHLC (high < max(open,close), low > min(open,close), low > high)

Then we filter to a clean set suitable for backtesting.

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Universe evaluation scope
INTERVALS = ['5m','15m','1h','4h','1d']
MIN_BARS = 2000
MAX_GAP_FRAC = 0.001  # allow 0.1% gaps
MAX_DUP_FRAC = 0.0001
MAX_INVALID_FRAC = 0.0001

INTERVAL_MS = {
  '1m': 60_000,
  '3m': 180_000,
  '5m': 300_000,
  '15m': 900_000,
  '30m': 1_800_000,
  '1h': 3_600_000,
  '2h': 7_200_000,
  '4h': 14_400_000,
  '6h': 21_600_000,
  '8h': 28_800_000,
  '12h': 43_200_000,
  '1d': 86_400_000,
}

interval_ms_expr = F.create_map([F.lit(x) for kv in INTERVAL_MS.items() for x in kv])

ohlc0 = (spark.table('ohlc')
  .where(F.col('interval').isin(INTERVALS))
  .select('exchange','symbol','interval','open_time','open','high','low','close','volume')
  .withColumn('expected_ms', interval_ms_expr[F.col('interval')])
)

w = Window.partitionBy('exchange','symbol','interval').orderBy('open_time')
ohlc1 = (ohlc0
  .withColumn('prev_open_time', F.lag('open_time').over(w))
  .withColumn('dt', F.col('open_time') - F.col('prev_open_time'))
  .withColumn('is_gap', (F.col('prev_open_time').isNotNull()) & (F.col('dt') != F.col('expected_ms')))
  .withColumn('is_dup', (F.col('prev_open_time').isNotNull()) & (F.col('dt') == F.lit(0)))
  .withColumn('invalid_ohlc', (F.col('low') > F.col('high')) | (F.col('high') < F.greatest(F.col('open'), F.col('close'))) | (F.col('low') > F.least(F.col('open'), F.col('close'))))
)

quality = (ohlc1
  .groupBy('exchange','symbol','interval','expected_ms')
  .agg(
    F.count('*').alias('bars'),
    F.sum(F.col('is_gap').cast('long')).alias('gap_bars'),
    F.sum(F.col('is_dup').cast('long')).alias('dup_bars'),
    F.sum(F.col('invalid_ohlc').cast('long')).alias('invalid_bars'),
    F.min('open_time').alias('min_open_time'),
    F.max('open_time').alias('max_open_time'),
  )
  .withColumn('gap_frac', F.col('gap_bars')/F.col('bars'))
  .withColumn('dup_frac', F.col('dup_bars')/F.col('bars'))
  .withColumn('invalid_frac', F.col('invalid_bars')/F.col('bars'))
  .orderBy(F.col('bars').desc())
)

display(quality.limit(200))


## Build and write `clean_universe`
The clean universe is the subset that passes our quality thresholds.

In [ ]:
clean_universe = (quality
  .where(F.col('bars') >= F.lit(MIN_BARS))
  .where(F.col('gap_frac') <= F.lit(MAX_GAP_FRAC))
  .where(F.col('dup_frac') <= F.lit(MAX_DUP_FRAC))
  .where(F.col('invalid_frac') <= F.lit(MAX_INVALID_FRAC))
  .withColumn('min_open_dt_utc', F.to_timestamp(F.col('min_open_time')/1000))
  .withColumn('max_open_dt_utc', F.to_timestamp(F.col('max_open_time')/1000))
  .withColumn('min_open_dt_syd', F.from_utc_timestamp(F.col('min_open_dt_utc'), 'Australia/Sydney'))
  .withColumn('max_open_dt_syd', F.from_utc_timestamp(F.col('max_open_dt_utc'), 'Australia/Sydney'))
)

spark.sql('''
CREATE TABLE IF NOT EXISTS clean_universe (
  exchange STRING,
  symbol STRING,
  interval STRING,
  expected_ms BIGINT,
  bars BIGINT,
  gap_bars BIGINT,
  dup_bars BIGINT,
  invalid_bars BIGINT,
  gap_frac DOUBLE,
  dup_frac DOUBLE,
  invalid_frac DOUBLE,
  min_open_time BIGINT,
  max_open_time BIGINT,
  min_open_dt_utc TIMESTAMP,
  max_open_dt_utc TIMESTAMP,
  min_open_dt_syd TIMESTAMP,
  max_open_dt_syd TIMESTAMP
) USING DELTA
''')

# Overwrite each time so downstream notebooks always reference latest clean set
(clean_universe
  .select('exchange','symbol','interval','expected_ms','bars','gap_bars','dup_bars','invalid_bars','gap_frac','dup_frac','invalid_frac',
          'min_open_time','max_open_time','min_open_dt_utc','max_open_dt_utc','min_open_dt_syd','max_open_dt_syd')
  .write
  .mode('overwrite')
  .option('overwriteSchema','true')
  .saveAsTable('clean_universe')
)

display(clean_universe.orderBy(F.col('bars').desc()).limit(200))
print('clean_universe count =', clean_universe.count())


## Quick OHLC preview with Sydney time

In [ ]:
%sql
SELECT
  exchange, symbol, interval,
  open_time,
  from_utc_timestamp(to_timestamp(open_time/1000), 'Australia/Sydney') AS open_time_syd,
  open, high, low, close, volume
FROM ohlc
ORDER BY open_time DESC
LIMIT 50;
